# Setup

In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
import seaborn as sns
import numpy as np
from functools import reduce

In [2]:
# There is some datacleanup needed as some counties are in all caps and listed as PA
def fix(df, inplace = True):
    if inplace:
        new_df = df
    else:
        new_df = df.copy()
        
    new_df["County"] = new_df["County"].map(lambda x: str.title(x.split(',')[0]))
    
    if not inplace:
        return new_df

## This Notebook combines all the county data 

# Reading Datasets

In [3]:
# Loading PA GOV
df = pd.read_csv('../data/PAGOV.csv')

# Loading Takeback dataset
takebackdf = pd.read_csv('../data/Prescription_Drug_Take-Back_Box_Locations_County_Drug_and_Alcohol_Programs.csv')

# Loading treatment
treatdf = pd.read_csv('../data/Drug_and_Alcohol_Treatment_Facilities_May_2018_County_Drug_and_Alcohol_Programs.csv')

# Risky Precribing dataset
risky_df = pd.read_csv('../data/Risky_Prescribing_Measures_Quarter_3_2016_-_Current_Quarterly_County___Statewide_Health.csv')

# Dispensation Dataset
dispen_df = pd.read_csv('../data/Dispensation_Data_without_Buprenorphine_Quarter_3_2016_-_Current_Quarterly_County_Health.csv')

# Loading Arrests dataset
arrests_df = pd.read_csv('../data/Opioid_Seizures_and_Arrests_CY_2013_-_Current_Quarterly_County_State_Police.csv')

# Aggregations

## PA GOV

In [4]:
###PA.gov
def yn_switch(op_df, inplace = True):
    
    if inplace:
        opdfnew = op_df
    else:
        opdfnew = op_df.copy()
        
    opdfnew["Narcan Admin"] = opdfnew['Naloxone Administered'].map(lambda x: 1 if x == "Y" else 0)
    opdfnew["Survive"] = opdfnew['Survive'].map(lambda x: 1 if x == "Y" else 0)
    
    if not inplace:
        return opdfnew

opiod_list = ['CARFENTANIL','FENTANYL','FENTANYL ANALOG/OTHER SYNTHETIC OPIOID','HEROIN','METHADONE','PHARMACEUTICAL OPIOID','SUBOXONE']

# Reduce incidents to ones with specific opioid
opdf = df.loc[df['Susp OD Drug Desc'].isin(opiod_list)]

# Store relevant information
opdf = opdf[['Incident ID','Incident County Name','Incident Date','Victim ID','Gender Desc','Day','Naloxone Administered','Age Range','Survive','Response Desc','Incident Time']]

# Remove duplicates
opdf = opdf.drop_duplicates()
opdf['Incident Date ym'] = pd.to_datetime(opdf['Incident Date']).dt.to_period('Y')

opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(str)
opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(int)

oppdf = opdf.groupby(['Incident Date ym'])[['Incident ID']].count().reset_index()
opdfnew = yn_switch(opdf, inplace = False)

# limiting
opdfnew = opdfnew[['Incident ID',
                   'Incident County Name',
                   'Narcan Admin',
                   'Survive',
                   'Incident Date ym']]

# Renaming for joins later
opdfnew  = opdfnew.rename(columns={'Incident County Name': 'County', 'Incident Date ym':'Year'})

# Getting totals
opdfnew['Total Overdoses Per County'] = opdfnew.groupby(['County','Year'])['County'].transform('count')
opdfnew['Percent Narcan Admin Per County/Year'] = opdfnew.groupby(['County','Year'])['Narcan Admin'].transform('mean')
opdfnew['Percent Survive Overdose Per County/Year'] = opdfnew.groupby(['County','Year'])['Survive'].transform('mean')

# Limiting
opdfnew = opdfnew[['County','Year',
                   'Percent Narcan Admin Per County/Year',
                   'Percent Survive Overdose Per County/Year']].drop_duplicates()


opdfnew.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year
2,Delaware,2018,0.656250,0.703125
4,Chester,2018,0.813397,0.751196
5,Beaver,2018,0.685185,0.814815
6,Bucks,2018,0.739895,0.804921
8,Philadelphia,2018,0.951220,0.926829


## Takeback and Treatments

In [5]:
# Next, Takeback and treaments
# Fixing County name
takebackdf1 = fix(takebackdf, False)

# Count takeback locations for each county
takebackdf1['Total of Take Back Locations by County'] = takebackdf1.groupby('County')['County'].transform('count')

# Count Treatment locations by County
treatdf['Total of Treatment Locations by County'] = treatdf.groupby('County')['County'].transform('count')

# Extract relevant columns
treatdf = treatdf[['County','Total of Treatment Locations by County']]

# Merging dataframes
dfcnt2 = pd.merge(takebackdf1,treatdf, how="inner",left_on = 'County',right_on = 'County')

dftakebackandtreatment = dfcnt2[['County','Total of Take Back Locations by County','Total of Treatment Locations by County']]
dftakebackandtreatment = dftakebackandtreatment.drop_duplicates()

# Merge Takeback dataframe with PA.GOV
dfmerge1 = pd.merge(opdfnew,dftakebackandtreatment, how="inner",left_on = 'County',right_on = 'County')
dfmerge1['Year'] = dfmerge1['Year'].astype(int)

# Sanity Check
dfmerge1.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County
0,Delaware,2018,0.656250,0.703125,55,30
1,Delaware,2019,0.717949,0.615385,55,30
2,Delaware,2020,0.555556,0.587302,55,30
3,Delaware,2021,0.560000,0.700000,55,30
4,Delaware,2022,0.500000,0.714286,55,30


## Risky Prescription

In [6]:
# Removing PA as to not count twice
risky_df = risky_df[risky_df['County'] != 'Pennsylvania']

# Suming per county/year
risky_df['Total Risky Prescibing per County/Year'] = risky_df.groupby(['County','Year'])['Rate or Count'].transform('sum')

# limiting
dfrisky = risky_df[['County','Year','Total Risky Prescibing per County/Year']].drop_duplicates()

dfrisky.head()

,County,Year,Total Risky Prescibing per County/Year
1,Lackawanna,2020,36468.040054
4,Mifflin,2021,3224.363865
5,Schuylkill,2021,11536.172907
6,Butler,2019,26160.090000
7,Snyder,2016,4034.040000


In [7]:
# Merging risky dataset
dfmerge2 = pd.merge(dfmerge1,dfrisky, how="inner",left_on = ['County','Year'],right_on = ['County','Year'])

dfmerge2.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year
0,Delaware,2018,0.656250,0.703125,55,30,87748.560000
1,Delaware,2019,0.717949,0.615385,55,30,70744.460000
2,Delaware,2020,0.555556,0.587302,55,30,57569.091674
3,Delaware,2021,0.560000,0.700000,55,30,38628.769756
4,Chester,2018,0.813397,0.751196,26,25,64356.480000


# Dispensation Data

In [8]:
# Keeping age groups, removing all ages 
# will have to do Genders seperatly as 
# They don't count them together
dispen_df = dispen_df[dispen_df['Age Group'] == 'All Ages']
dispen_df = dispen_df[dispen_df['Gender'] == 'All Genders']

dispen_df  = dispen_df.rename(columns={'County Name': 'County'})
dispen_df['Total Dispensation per County/Year'] = dispen_df.groupby(['County','Year'])['Rate or Count'].transform('sum')
dispdf = dispen_df[['County','Year','Total Dispensation per County/Year']].drop_duplicates()

##Merging Dispensation
dfmerge3 = pd.merge(dfmerge2,dispdf, how="inner",left_on = ['County','Year'],right_on = ['County','Year']).drop_duplicates()
dfmerge3.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year,Total Dispensation per County/Year
0,Delaware,2018,0.656250,0.703125,55,30,87748.560000,653144.920000
1,Delaware,2019,0.717949,0.615385,55,30,70744.460000,582654.380000
2,Delaware,2020,0.555556,0.587302,55,30,57569.091674,514813.120000
3,Delaware,2021,0.560000,0.700000,55,30,38628.769756,367033.862535
4,Chester,2018,0.813397,0.751196,26,25,64356.480000,480173.990000


# Arrests

In [9]:
arrests_df = arrests_df.rename(columns={'County Name': 'County'})

df1Fentanyl = arrests_df.assign(Fentanyl_Incident = np.where(arrests_df['Drug']=='Fentanyl',arrests_df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Incident':sum}).reset_index().drop_duplicates()

df1Heroin= arrests_df.assign(Heroin_Incident = np.where(arrests_df['Drug']=='Heroin',arrests_df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Heroin_Incident':sum}).reset_index().drop_duplicates()

df1Opium= arrests_df.assign(Opium_Incident = np.where(arrests_df['Drug']=='Opium',arrests_df['Incident Count'],0))\
   .groupby(['County','Year']).agg({'Opium_Incident':sum}).reset_index().drop_duplicates()


df2Fentanyl = arrests_df.assign(Fentanyl_Arrests = np.where(arrests_df['Drug']=='Fentanyl',arrests_df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Arrests':sum}).reset_index().drop_duplicates()

df2Heroin= arrests_df.assign(Heroin_Arrests = np.where(arrests_df['Drug']=='Heroin',arrests_df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Heroin_Arrests':sum}).reset_index().drop_duplicates()

df2Opium= arrests_df.assign(Opium_Arrests = np.where(arrests_df['Drug']=='Opium',arrests_df['Arrests'],0))\
   .groupby(['County','Year']).agg({'Opium_Arrests':sum}).reset_index().drop_duplicates()


df3Fentanyl = arrests_df.assign(Fentanyl_Quantity = np.where(arrests_df['Drug']=='Fentanyl',arrests_df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Fentanyl_Quantity':sum}).reset_index().drop_duplicates()

df3Heroin= arrests_df.assign(Heroin_Quantity = np.where(arrests_df['Drug']=='Heroin',arrests_df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Heroin_Quantity':sum}).reset_index().drop_duplicates()

df3Opium= arrests_df.assign(Opium_Quantity = np.where(arrests_df['Drug']=='Opium',arrests_df['Drug Quantity'],0))\
   .groupby(['County','Year']).agg({'Opium_Quantity':sum}).reset_index().drop_duplicates()


In [10]:
data_frames = [df1Fentanyl,df1Heroin,df1Opium,df2Fentanyl,df2Heroin,df2Opium,df3Fentanyl,
              df3Heroin,df3Opium]  ## combining dataframes

# Merging dataframe
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['County','Year'],
                                            how='outer'), data_frames)

# Creating computed collumns
df_merged['%Incidents Fentanyl'] = (df_merged['Fentanyl_Incident']/ 
(df_merged['Fentanyl_Incident'] +df_merged['Opium_Incident'] + df_merged['Heroin_Incident']*100))

df_merged['%Arrests Fentanyl'] = (df_merged['Fentanyl_Arrests']/ 
(df_merged['Fentanyl_Arrests'] + df_merged['Opium_Arrests'] + df_merged['Heroin_Arrests']*100))

df_merged['%Quantity Fentanyl'] = (df_merged['Fentanyl_Quantity']/ 
(df_merged['Fentanyl_Quantity'] + df_merged['Opium_Quantity'] + df_merged['Heroin_Quantity']*100))
dfarrestsfinal = df_merged

In [11]:
dfmerge4 = pd.merge(dfmerge3,dfarrestsfinal, how="inner",left_on = ['County','Year'],right_on = ['County','Year']).drop_duplicates()
dfmerge4.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year,Total Dispensation per County/Year,Fentanyl_Incident,Heroin_Incident,Opium_Incident,Fentanyl_Arrests,Heroin_Arrests,Opium_Arrests,Fentanyl_Quantity,Heroin_Quantity,Opium_Quantity,%Incidents Fentanyl,%Arrests Fentanyl,%Quantity Fentanyl
0,Delaware,2018,0.656250,0.703125,55,30,87748.560000,653144.920000,5,144,0,3,164,0,0.02077,0.538028,0.000000,0.000347,0.000183,0.000386
1,Delaware,2019,0.717949,0.615385,55,30,70744.460000,582654.380000,15,173,0,19,198,0,5.27221,5.501150,0.000000,0.000866,0.000959,0.009493
2,Delaware,2020,0.555556,0.587302,55,30,57569.091674,514813.120000,23,159,0,27,202,0,0.16478,1.270373,0.000000,0.001444,0.001335,0.001295
3,Delaware,2021,0.560000,0.700000,55,30,38628.769756,367033.862535,27,122,2,39,150,2,17.05824,4.397683,0.000012,0.002208,0.002593,0.037341
4,Chester,2018,0.813397,0.751196,26,25,64356.480000,480173.990000,5,76,2,6,64,2,1.09655,0.106369,0.000250,0.000657,0.000936,0.093453
